In [122]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [123]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
import random

In [ ]:
def plot_columns(df):
    colors = plt.cm.tab20(range(df.shape[1]))  

    for i, column in enumerate(df.columns):
        color = colors[i % 24]

        plt.figure()
        plt.plot(df.index, df[column], color=color)
        plt.xlabel('Tempo')
        plt.ylabel('Valor')
        plt.title(column)

    plt.show()

In [ ]:
def plot_correlation_matrix(df):
    correlation_matrix = df.corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title("Correlation Matrix Heatmap")
    plt.show()

In [124]:
tf.keras.backend.clear_session()

In [125]:
subject = 'Pará - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [126]:
data = pd.read_csv('2003_mo_model_input_MG.csv')
pd.set_option('display.max_columns', None)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - Desemprego,Pará - Produção de Cimento (t),Pará - value,Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Pará - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.348543,49.618560,0.312867,0.669564,7.776091e+07,4.250430e+06,9.220374,6.851269e+07,84.701
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.342554,49.658098,0.312657,0.669764,7.787181e+07,4.255476e+06,9.225493,6.856365e+07,80.221
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.336566,49.906876,0.311736,0.669963,7.798271e+07,4.260521e+06,9.230612,6.861461e+07,76.122
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.330578,49.864305,0.310493,0.670163,7.809361e+07,4.265566e+06,9.235731,6.866557e+07,76.544
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.324589,50.179753,0.309128,0.670362,7.820451e+07,4.270611e+06,9.240851,6.871652e+07,83.445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.899018,0.710173,NaN,NaN,NaN,NaN,NaN,156.442
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.416567,0.708384,NaN,NaN,NaN,NaN,NaN,145.777
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.559167,0.706038,NaN,NaN,NaN,NaN,NaN,139.233
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.696005,0.702147,NaN,NaN,NaN,NaN,NaN,142.025


In [ ]:
def calculate_monthly_means(consume_of_cement, subject):
    df = pd.DataFrame({subject: consume_of_cement[:train_split]})
    df['sazonalidade'] = df.index % 12
    monthly_means = df.groupby('sazonalidade')[subject].mean()
    result_df = pd.DataFrame({'sazonalidade': monthly_means})

    return result_df

media_df = calculate_monthly_means(data[subject], subject)
plot_columns(media_df)
media_df

In [ ]:
def fill_original_df_with_monthly_means(df_og, media_df):
    monthly_means = media_df['sazonalidade'].tolist()
    df_og['sazonalidade'] = [monthly_means[i % 12] for i in range(len(df_og))]
    return df_og

data = fill_original_df_with_monthly_means(data, media_df)
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

In [ ]:
plot_data = data.iloc[:split_index + 1,1:]
plot_correlation_matrix(plot_data)

In [ ]:
# data =data.drop(['Air Temperature - Dry Bulb (°C)',
#                  'Precipitation (mm/day)',
#                  'Global Radiation - Daylight (Kj/m²)', 
#                  'Air Relative Humidity (%)'], axis=1)
# data

In [127]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
# scaler=MinMaxScaler()
# scaled_data = scaler.fit_transform(input_data)
# input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - Desemprego,Pará - Produção de Cimento (t),Pará - value,Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.743008,-1.860097,-1.001954,-1.906833,-1.652592,-1.810463,-2.287151,-1.950096
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.748918,-1.858558,-1.004769,-1.880912,-1.635387,-1.758101,-2.245800,-1.920309
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.754829,-1.848873,-1.017098,-1.854991,-1.618182,-1.705740,-2.204450,-1.890522
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.760740,-1.850531,-1.033739,-1.829071,-1.600977,-1.653378,-2.163099,-1.860734
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.766650,-1.838250,-1.052007,-1.803150,-1.583772,-1.601017,-2.121748,-1.830947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.076246,-0.644020,1.421704,1.389768,1.316401,-0.980846,1.292832,1.315283
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.073514,-0.676398,1.447588,1.378488,1.314484,-0.973932,1.302757,1.318820
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.070782,-0.703221,1.485640,1.367208,1.312566,-0.967018,1.312682,1.322356
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.068050,-0.725198,1.528780,1.355927,1.310648,-0.960105,1.322607,1.325893


In [128]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      110.204
1       76.354
2       77.171
3       80.720
4       82.088
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Pará - Consumo de Cimento (t), Length: 240, dtype: float64

In [129]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - Desemprego,Pará - Produção de Cimento (t),Pará - value,Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.743008,-1.860097,-1.001954,-1.906833,-1.652592,-1.810463,-2.287151,-1.950096
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.748918,-1.858558,-1.004769,-1.880912,-1.635387,-1.758101,-2.245800,-1.920309
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.754829,-1.848873,-1.017098,-1.854991,-1.618182,-1.705740,-2.204450,-1.890522
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.760740,-1.850531,-1.033739,-1.829071,-1.600977,-1.653378,-2.163099,-1.860734
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.766650,-1.838250,-1.052007,-1.803150,-1.583772,-1.601017,-2.121748,-1.830947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.444243,0.582557,1.197817,1.368533,1.186547,-1.094003,0.861099,1.054711
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.421118,0.538378,1.208551,1.376125,1.197116,-1.099823,0.885880,1.071769
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.397993,0.482165,1.218732,1.383716,1.207686,-1.105643,0.910662,1.088827
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.374869,0.427013,1.224310,1.391308,1.218256,-1.111463,0.935444,1.105885


In [130]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      110.204
1       76.354
2       77.171
3       80.720
4       82.088
        ...   
157    116.872
158    131.959
159    116.171
160    132.242
161    135.615
Name: Pará - Consumo de Cimento (t), Length: 162, dtype: float64

In [131]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [132]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [133]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - Desemprego,Pará - Produção de Cimento (t),Pará - value,Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
126,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.937142,1.114270,0.390386,0.491346,0.718639,0.747580,0.575976,0.685697
127,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.968970,1.082022,0.414807,0.506090,0.735578,0.698813,0.571078,0.692184
128,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,1.000798,1.052039,0.439827,0.520835,0.752516,0.650046,0.566181,0.698671
129,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,1.032625,1.020267,0.471477,0.535579,0.769455,0.601279,0.561283,0.705157
130,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,1.064453,0.982058,0.505972,0.550323,0.786394,0.552512,0.556385,0.711644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.076246,-0.644020,1.421704,1.389768,1.316401,-0.980846,1.292832,1.315283
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.073514,-0.676398,1.447588,1.378488,1.314484,-0.973932,1.302757,1.318820
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.070782,-0.703221,1.485640,1.367208,1.312566,-0.967018,1.312682,1.322356
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.068050,-0.725198,1.528780,1.355927,1.310648,-0.960105,1.322607,1.325893


In [134]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [135]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [136]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
#     optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(72, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(36,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')    
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [137]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [138]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[782675668, 1311471000, 1351225546, 498673338, 1595719333, 3118980657, 2135154548, 744231899, 942361708, 4137334450]


Step: 0 ___________________________________________
val_loss: 83.73699188232422
winner_seed: 782675668


Step: 1 ___________________________________________
val_loss: 106.95693969726562


Step: 2 ___________________________________________
val_loss: 100.50688934326172


Step: 3 ___________________________________________
val_loss: 102.01081085205078


Step: 4 ___________________________________________
val_loss: 81.37228393554688
winner_seed: 1595719333


Step: 5 ___________________________________________
val_loss: 88.24652862548828


Step: 6 ___________________________________________
val_loss: 83.03154754638672


Step: 7 ___________________________________________
val_loss: 114.04911041259766


Step: 8 ___________________________________________
val_loss: 111.78329467773438


Step: 9 ___________________________________________
val_loss: 88.50182342529297


final_see

In [139]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)
# 1595719333

Epoch 1/10000
4/4 [==============================] - 3s 121ms/step - loss: 24962.3926 - val_loss: 25541.5039
Epoch 2/10000
4/4 [==============================] - 0s 9ms/step - loss: 24723.8516 - val_loss: 25275.0684
Epoch 3/10000
4/4 [==============================] - 0s 10ms/step - loss: 24470.4961 - val_loss: 24976.6816
Epoch 4/10000
4/4 [==============================] - 0s 10ms/step - loss: 24190.0508 - val_loss: 24666.5059
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 23885.7227 - val_loss: 24358.4844
Epoch 6/10000
4/4 [==============================] - 0s 9ms/step - loss: 23575.7676 - val_loss: 24048.5820
Epoch 7/10000
4/4 [==============================] - 0s 8ms/step - loss: 23276.3359 - val_loss: 23756.1719
Epoch 8/10000
4/4 [==============================] - 0s 8ms/step - loss: 22987.9375 - val_loss: 23493.3711
Epoch 9/10000
4/4 [==============================] - 0s 8ms/step - loss: 22725.7012 - val_loss: 23249.6035
Epoch 10/10000
4/4 [=============

Epoch 77/10000
4/4 [==============================] - 0s 8ms/step - loss: 18934.4316 - val_loss: 19567.2715
Epoch 78/10000
4/4 [==============================] - 0s 9ms/step - loss: 18894.7148 - val_loss: 19526.8848
Epoch 79/10000
4/4 [==============================] - 0s 8ms/step - loss: 18855.2324 - val_loss: 19486.5703
Epoch 80/10000
4/4 [==============================] - 0s 8ms/step - loss: 18815.1738 - val_loss: 19446.5078
Epoch 81/10000
4/4 [==============================] - 0s 9ms/step - loss: 18776.3828 - val_loss: 19406.1387
Epoch 82/10000
4/4 [==============================] - 0s 9ms/step - loss: 18736.4082 - val_loss: 19365.7148
Epoch 83/10000
4/4 [==============================] - 0s 10ms/step - loss: 18696.5215 - val_loss: 19325.3848
Epoch 84/10000
4/4 [==============================] - 0s 9ms/step - loss: 18656.4863 - val_loss: 19285.1465
Epoch 85/10000
4/4 [==============================] - 0s 9ms/step - loss: 18617.5957 - val_loss: 19245.1660
Epoch 86/10000
4/4 [=======

4/4 [==============================] - 0s 9ms/step - loss: 13764.3555 - val_loss: 14309.4961
Epoch 228/10000
4/4 [==============================] - 0s 8ms/step - loss: 13734.2754 - val_loss: 14279.1123
Epoch 229/10000
4/4 [==============================] - 0s 8ms/step - loss: 13704.4131 - val_loss: 14248.7402
Epoch 230/10000
4/4 [==============================] - 0s 9ms/step - loss: 13674.7773 - val_loss: 14218.6025
Epoch 231/10000
4/4 [==============================] - 0s 9ms/step - loss: 13645.2998 - val_loss: 14188.6611
Epoch 232/10000
4/4 [==============================] - 0s 9ms/step - loss: 13615.7666 - val_loss: 14158.7236
Epoch 233/10000
4/4 [==============================] - 0s 10ms/step - loss: 13586.7354 - val_loss: 14128.7725
Epoch 234/10000
4/4 [==============================] - 0s 9ms/step - loss: 13557.1836 - val_loss: 14098.9277
Epoch 235/10000
4/4 [==============================] - 0s 9ms/step - loss: 13527.9434 - val_loss: 14069.1572
Epoch 236/10000
4/4 [=============

4/4 [==============================] - 0s 9ms/step - loss: 9819.1836 - val_loss: 10282.5742
Epoch 378/10000
4/4 [==============================] - 0s 9ms/step - loss: 9796.2422 - val_loss: 10259.3721
Epoch 379/10000
4/4 [==============================] - 0s 9ms/step - loss: 9773.5869 - val_loss: 10236.1016
Epoch 380/10000
4/4 [==============================] - 0s 9ms/step - loss: 9750.9062 - val_loss: 10213.0254
Epoch 381/10000
4/4 [==============================] - 0s 9ms/step - loss: 9728.3125 - val_loss: 10189.8828
Epoch 382/10000
4/4 [==============================] - 0s 8ms/step - loss: 9705.3418 - val_loss: 10166.6797
Epoch 383/10000
4/4 [==============================] - 0s 8ms/step - loss: 9682.5938 - val_loss: 10143.1914
Epoch 384/10000
4/4 [==============================] - 0s 8ms/step - loss: 9660.1328 - val_loss: 10119.5576
Epoch 385/10000
4/4 [==============================] - 0s 8ms/step - loss: 9637.0527 - val_loss: 10096.3838
Epoch 386/10000
4/4 [=======================

4/4 [==============================] - 0s 9ms/step - loss: 6807.3560 - val_loss: 7194.0820
Epoch 530/10000
4/4 [==============================] - 0s 9ms/step - loss: 6790.6602 - val_loss: 7176.6064
Epoch 531/10000
4/4 [==============================] - 0s 9ms/step - loss: 6773.5542 - val_loss: 7159.2866
Epoch 532/10000
4/4 [==============================] - 0s 9ms/step - loss: 6756.9688 - val_loss: 7141.9590
Epoch 533/10000
4/4 [==============================] - 0s 10ms/step - loss: 6740.0737 - val_loss: 7124.8267
Epoch 534/10000
4/4 [==============================] - 0s 10ms/step - loss: 6723.5054 - val_loss: 7107.6528
Epoch 535/10000
4/4 [==============================] - 0s 11ms/step - loss: 6706.5562 - val_loss: 7090.6514
Epoch 536/10000
4/4 [==============================] - 0s 9ms/step - loss: 6690.0630 - val_loss: 7073.5581
Epoch 537/10000
4/4 [==============================] - 0s 8ms/step - loss: 6673.6284 - val_loss: 7056.4360
Epoch 538/10000
4/4 [=============================

...

4/4 [==============================] - 0s 8ms/step - loss: 59.6509 - val_loss: 159.0433
Epoch 2905/10000
4/4 [==============================] - 0s 8ms/step - loss: 59.7937 - val_loss: 163.9966
Epoch 2906/10000
4/4 [==============================] - 0s 8ms/step - loss: 71.0796 - val_loss: 156.5486
Epoch 2907/10000
4/4 [==============================] - 0s 8ms/step - loss: 63.1238 - val_loss: 145.0394
Epoch 2908/10000
4/4 [==============================] - 0s 8ms/step - loss: 61.2563 - val_loss: 138.3069
Epoch 2909/10000
4/4 [==============================] - 0s 8ms/step - loss: 62.0685 - val_loss: 136.1075
Epoch 2910/10000
4/4 [==============================] - 0s 8ms/step - loss: 61.6656 - val_loss: 139.0540
Epoch 2911/10000
4/4 [==============================] - 0s 8ms/step - loss: 62.6898 - val_loss: 140.6317
Epoch 2912/10000
4/4 [==============================] - 0s 8ms/step - loss: 58.3024 - val_loss: 141.7027
Epoch 2913/10000
4/4 [==============================] - 0s 8ms/step - lo

4/4 [==============================] - 0s 10ms/step - loss: 55.1220 - val_loss: 139.0250
Epoch 3061/10000
4/4 [==============================] - 0s 9ms/step - loss: 54.6654 - val_loss: 146.6645
Epoch 3062/10000
4/4 [==============================] - 0s 9ms/step - loss: 59.6149 - val_loss: 149.9592
Epoch 3063/10000
4/4 [==============================] - 0s 10ms/step - loss: 58.1808 - val_loss: 149.6471
Epoch 3064/10000
4/4 [==============================] - 0s 9ms/step - loss: 60.2606 - val_loss: 150.7045
Epoch 3065/10000
4/4 [==============================] - 0s 9ms/step - loss: 56.2111 - val_loss: 151.9133
Epoch 3066/10000
4/4 [==============================] - 0s 9ms/step - loss: 50.4537 - val_loss: 152.9061
Epoch 3067/10000
4/4 [==============================] - 0s 9ms/step - loss: 58.7301 - val_loss: 153.2665
Epoch 3068/10000
4/4 [==============================] - 0s 9ms/step - loss: 53.7889 - val_loss: 146.0033
Epoch 3069/10000
4/4 [==============================] - 0s 9ms/step - 

4/4 [==============================] - 0s 9ms/step - loss: 57.8947 - val_loss: 159.3537
Epoch 3139/10000
4/4 [==============================] - 0s 9ms/step - loss: 57.0165 - val_loss: 165.8098
Epoch 3140/10000
4/4 [==============================] - 0s 9ms/step - loss: 47.5669 - val_loss: 170.6443
Epoch 3141/10000
4/4 [==============================] - 0s 9ms/step - loss: 51.6895 - val_loss: 161.4098
Epoch 3142/10000
4/4 [==============================] - 0s 9ms/step - loss: 57.2353 - val_loss: 151.5052
Epoch 3143/10000
4/4 [==============================] - 0s 9ms/step - loss: 48.8368 - val_loss: 149.9470
Epoch 3144/10000
4/4 [==============================] - 0s 9ms/step - loss: 56.9356 - val_loss: 153.7300
Epoch 3145/10000
4/4 [==============================] - 0s 9ms/step - loss: 59.6555 - val_loss: 157.8186
Epoch 3146/10000
4/4 [==============================] - 0s 10ms/step - loss: 53.4257 - val_loss: 156.8568
Epoch 3147/10000
4/4 [==============================] - 0s 8ms/step - l

4/4 [==============================] - 0s 9ms/step - loss: 62.8109 - val_loss: 159.2982
Epoch 3217/10000
4/4 [==============================] - 0s 9ms/step - loss: 64.0022 - val_loss: 159.2839
Epoch 3218/10000
4/4 [==============================] - 0s 9ms/step - loss: 71.2154 - val_loss: 155.3320
Epoch 3219/10000
4/4 [==============================] - 0s 10ms/step - loss: 59.7358 - val_loss: 144.3879
Epoch 3220/10000
4/4 [==============================] - 0s 9ms/step - loss: 60.2393 - val_loss: 136.3049
Epoch 3221/10000
4/4 [==============================] - 0s 9ms/step - loss: 55.4775 - val_loss: 139.4865
Epoch 3222/10000
4/4 [==============================] - 0s 9ms/step - loss: 57.0005 - val_loss: 141.0898
Epoch 3223/10000
4/4 [==============================] - 0s 9ms/step - loss: 57.6493 - val_loss: 154.7072
Epoch 3224/10000
4/4 [==============================] - 0s 9ms/step - loss: 53.6459 - val_loss: 156.3450
Epoch 3225/10000
4/4 [==============================] - 0s 9ms/step - l

4/4 [==============================] - 0s 8ms/step - loss: 51.8751 - val_loss: 153.6341
Epoch 3295/10000
4/4 [==============================] - 0s 9ms/step - loss: 49.4468 - val_loss: 149.5245
Epoch 3296/10000
4/4 [==============================] - 0s 9ms/step - loss: 54.3934 - val_loss: 149.6970
Epoch 3297/10000
4/4 [==============================] - 0s 9ms/step - loss: 54.1510 - val_loss: 151.6432
Epoch 3298/10000
4/4 [==============================] - 0s 9ms/step - loss: 52.8663 - val_loss: 156.8757
Epoch 3299/10000
4/4 [==============================] - 0s 9ms/step - loss: 51.6478 - val_loss: 163.8472
Epoch 3300/10000
4/4 [==============================] - 0s 10ms/step - loss: 52.9542 - val_loss: 172.0918
Epoch 3301/10000
4/4 [==============================] - 0s 9ms/step - loss: 53.8529 - val_loss: 171.1877
Epoch 3302/10000
4/4 [==============================] - 0s 9ms/step - loss: 50.8983 - val_loss: 167.9467
Epoch 3303/10000
4/4 [==============================] - 0s 8ms/step - l

In [140]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [141]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,127.988548,127.97995,127.961693,127.916405,127.876854,127.84214,127.825645,127.797005,127.744682,127.692398,127.689537,127.716278,127.697327,127.726471,127.715889,127.693512,127.647949,127.612335,127.63678,127.630577,127.614609,127.592339,127.519554,127.385574,127.30365,127.259888,127.216232,127.163834,127.150047,127.177498
Target,136.117,136.475,125.3,132.627,135.172,131.376,111.175,80.97,92.173,86.782,85.15,135.094,132.351,133.744,128.527,147.51,138.511,103.294,110.704,91.638,90.744,82.108,97.49,96.803,118.687,109.761,120.188,131.62,125.374,115.811
Error,8.128456,8.495056,2.66169,4.710594,7.295143,3.533867,16.650642,46.827003,35.571686,40.9104,42.539536,7.377716,4.653671,6.017532,0.811104,19.816483,10.863052,24.318336,16.932777,35.992577,36.870605,45.484337,30.029556,30.582573,8.616653,17.498886,7.028229,4.456161,1.776047,11.366501


In [142]:
display(mae)
display(mape)

17.927229

0.18335012

In [143]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [144]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[1534.0311]] - Target[1388.411]| =  Error: [[145.62012]]; MAPE:[[0.10488257]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[1531.4729]] - Target[1353.424]| =  Error: [[178.04895]]; MAPE:[[0.13155445]]
1/1 [==============================] - 0s 15ms/step
Ano-5: |Prediction[[763.2711]] - Target[721.441]| =  Error: [[41.83014]]; MAPE:[[0.05798138]]


[array([[145.62012]], dtype=float32),
 array([[178.04895]], dtype=float32),
 array([[41.83014]], dtype=float32)]

121.83307

0.098139465